# 安装
```shell
pip install ragas
```

# 1. AspectCritic

通过AspectCritic，设置提示词，可以灵活地让LLM做出任何评判。

参数：
- name: str, 名称
- definition: str, 定义需要LLM做什么！
- llm: BaseRagasLLM, 大语言模型实例



```python
from ragas.metrics import AspectCritic
```


# 2. 样本
单论样本， SingleTurnSample

- user_input: str, 用户输入
- retrieved_contexts: List[str], 检索到的上下文
- reference_contexts: List[str], 参考上下文
- response: str, 响应/回答
- multi_responses: List[str], 多个响应
- reference: str, 参考答案
- rubrics: Dict[str, str] ， 评分标准 ？？？

多论样本, MultiTurnSample

- user_input: str, 用户输入
- reference: str, 参考答案
- reference_tool_calls: List[ToolCall]
- rubrics: Dict[str, str] ， 评分标准 ？？？
- reference_topics: List[str] ????


In [1]:
import json

# 原始数据
line = '{"id": "968d0465-4d3f-4b5c-8326-58cecc4aa70c", "question": "\\u6839\\u636e\\u80cc\\u666f\\u4fe1\\u606f\\uff0c2025\\u5e74\\u79d1\\u6280\\u884c\\u4e1a\\u5c55\\u671b\\u4e2dAI\\u6d6a\\u6f6e\\u91cd\\u65b0\\u5b9a\\u4e49\\u4e86\\u5168\\u7403\\u79d1\\u6280\\u7684\\u672a\\u6765\\uff0c\\u8bf7\\u95ee\\u57282024\\u5e7412\\u670813\\u65e5\\u8fd9\\u4efd\\u62a5\\u544a\\u4e2d\\u6709\\u54ea\\u51e0\\u4f4d\\u4f5c\\u8005\\uff1f"}'

# 解析 JSON 数据
data = json.loads(line)
data

{'id': '968d0465-4d3f-4b5c-8326-58cecc4aa70c',
 'question': '根据背景信息，2025年科技行业展望中AI浪潮重新定义了全球科技的未来，请问在2024年12月13日这份报告中有哪几位作者？'}

In [2]:
# 我们把id的文档片段也拿出来一下
import chromadb
client = chromadb.HttpClient(host="localhost", port=8000)
collection = client.get_or_create_collection(name="rag-video-collection")
doc = collection.get(ids=["968d0465-4d3f-4b5c-8326-58cecc4aa70c"])
doc

{'ids': ['968d0465-4d3f-4b5c-8326-58cecc4aa70c'],
 'embeddings': None,
 'metadatas': [{'source': '科技行业 2025 年展望.txt',
   '作者': '沈岱,马智焱,黄佳琦',
   '发表时间': '2024 年 12 月 13 日',
   '文章标题': '科技行业2025年展望'}],
 'documents': ['科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间'],
 'data': None,
 'uris': None,
 'included': ['documents', 'metadatas']}

In [6]:
data = {
    "page_content": "科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间",
    "question": "根据背景信息，2025年科技行业展望中AI浪潮重新定义了全球科技的未来，请问在2024年12月13日这份报告中有哪几位作者？"
}

In [12]:
data = {
    "page_content": "重要的事，通常不紧急，紧急的事，通常不重要。要完成重要的事，就不要操之过急。",
    "question": "根据背景信息，2025年科技行业展望中AI浪潮重新定义了全球科技的未来，请问在2024年12月13日这份报告中有哪几位作者？"
}

In [13]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic
from langchain_ollama import ChatOllama
from ragas.llms import LangchainLLMWrapper

llm = ChatOllama(
        model="qwen2.5:latest",
    )

# metric = AspectCritic(name="relevance_assessment", 
#                       llm=LangchainLLMWrapper(llm), 
#                       definition="判断问题和文档内容是否相关？")
metric = AspectCritic(name="relevance_assessment", 
                      llm=LangchainLLMWrapper(llm), 
                      definition="判断输入(user_input)和上下文(retrieved_contexts)是否相关？")
test_data = SingleTurnSample(
    user_input = data["question"],
    retrieved_contexts = [data["page_content"]]
)
await metric.single_turn_ascore(test_data)

0

AspectCritic 提示词：
Evaluate the Input based on the criterial defined. Use only 'Yes' (1) and 'No' (0) as verdict.
Criteria Definition: 判断输入(user_input)和上下文(retrieved_contexts)是否相关？
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"reason": {"description": "Reason for the verdict", "title": "Reason", "type": "string"}, "verdict": {"description": "The verdict (0 or 1) for the submission", "title": "Verdict", "type": "integer"}}, "required": ["reason", "verdict"], "title": "AspectCriticOutput", "type": "object"}
Do not use single quotes in your response but double quotes,properly escaped with a backslash.

-----------------------------

Now perform the same with the following input
input: {
    "user_input": "根据背景信息，2025年科技行业展望中AI浪潮重新定义了全球科技的未来，请问在2024年12月13日这份报告中有哪几位作者？",
    "retrieved_contexts": [
        "科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间"
    ]
}
Output: 

# 3 批量评估

```python
from ragas import evaluate
```

evalute()方法的参数：

- dataset: Dataset | EvaluationDataset,
- metrics: List[Metric]
- llm: BaseRagasLLM, 大语言模型实例

